# Week 08 — Practical A/B Test Design (실무 실험 설계)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 실험 목적/가설/KPI/가드레일을 명확히 정의할 수 있는가?
- 샘플 수가 충분한지(대략) 판단할 수 있는가?
- 실험 결과를 '확대/중단/보완' 중 무엇으로 결정할 것인가?

## 2) Why this matters (Theory)
- 실험은 분석보다 설계가 80% (KPI/가드레일/대상/기간)
- 가드레일(마진/환불/CS) 없으면 잘못된 승리 선언 가능
- 샘플 수는 작은 uplift를 잡으려면 매우 커짐

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 A/B one-page template (fill this)
✅ 아래 변수를 채워서 '실험 문서'로 사용하세요.

In [ ]:
ab_onepager = {
    "Objective": "e.g., Increase conversion rate for Ecommerce traffic",
    "Hypothesis_H1": "Campaign A increases conversion rate vs B",
    "Primary_KPI": "Conversion rate",
    "Secondary_KPI": "Revenue per user",
    "Guardrail_KPI": "Gross margin per user",
    "Population": "e.g., KR + Ecommerce visitors",
    "Randomization": "user-level random",
    "Duration_days": 14,
    "Decision_rule": "e.g., uplift>2% and guardrail not worse"
}
ab_onepager

In [ ]:
one_sentence_insight('이 실험에서 절대 깨지면 안 되는 가드레일 KPI를 한 문장')

### 5.2 Quick sample size intuition (approx)
✅ 정교한 power 계산이 아니라, 실무 감각용 근사입니다.

In [ ]:
# Approx sample size per group for proportion difference (very rough)
# n ≈ 16 * p*(1-p) / (delta^2)
p = df['conversions'].mean()          # baseline conv
delta = 0.01                         # want to detect +1%p absolute lift
n = 16 * p*(1-p) / (delta**2)
print("baseline p:", p)
print("rough n per group to detect +1%p:", int(np.ceil(n)))

In [ ]:
one_sentence_insight('원하는 uplift를 잡기 위해 표본이 충분한지(기간/트래픽 관점) 한 문장')

### 5.3 Reporting format: result table skeleton
✅ 실험 결과를 일관된 표로 정리합니다.

In [ ]:
def ab_result_table(df_in, group_col='campaign', groups=('A','B')):
    g1, g2 = groups
    s1 = df_in[df_in[group_col]==g1]['conversions']
    s2 = df_in[df_in[group_col]==g2]['conversions']
    p1, p2 = s1.mean(), s2.mean()
    uplift = (p1/p2 - 1) if p2>0 else np.nan

    # revenue and margin bootstrap
    r1 = df_in[df_in[group_col]==g1]['revenue']
    r2 = df_in[df_in[group_col]==g2]['revenue']
    rd, rci = bootstrap_mean_diff(r1, r2, iters=3000, seed=0)

    m1 = df_in[df_in[group_col]==g1]['gross_margin']
    m2 = df_in[df_in[group_col]==g2]['gross_margin']
    md, mci = bootstrap_mean_diff(m1, m2, iters=3000, seed=1)

    return pd.DataFrame([{
        "group1": g1, "group2": g2,
        "conv_rate_1": p1, "conv_rate_2": p2,
        "uplift": uplift,
        "rev_mean_diff": rd, "rev_ci_low": rci[0], "rev_ci_high": rci[1],
        "margin_mean_diff": md, "margin_ci_low": mci[0], "margin_ci_high": mci[1],
        "n1": len(s1), "n2": len(s2)
    }])

ab = df[df['campaign'].isin(['A','B'])].copy()
display(ab_result_table(ab, group_col='campaign', groups=('A','B')))

In [ ]:
one_sentence_insight('전환/매출/마진을 종합한 최종 의사결정을 한 문장')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
